In [1]:
import collections

import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
import tensorflow_federated as tff
import pandas as pd
from sklearn.model_selection import train_test_split

TEST_SIZE = 0.2
NUM_CLIENTS = 5
ACTIVE_CLIENTS = 5
BATCH_SIZE = 5
path = os.path.dirname(tff.__file__)
print(path)

2024-09-02 12:57:35.090463: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-02 12:57:35.090577: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-02 12:57:35.092433: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-02 12:57:35.300477: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


/home/ella/Desktop/Progetto/venv-federated/lib/python3.9/site-packages/tensorflow_federated


In [2]:
# Lista delle GPU disponibili
gpus = tf.config.list_physical_devices('GPU')
print("GPUs disponibili: ", gpus)

# Verifica se TensorFlow utilizza la GPU
if gpus:
    print("TensorFlow sta usando la GPU")
else:
    print("TensorFlow non sta usando la GPU")

GPUs disponibili:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow sta usando la GPU


In [3]:
# Import del dataset e divisione in train e test
df = pd.read_csv('comptagesvelo2015.csv')
train_df, test_df = train_test_split(df, test_size = TEST_SIZE, random_state = 42)

# Funzione per il preprocessiSng dei dati del singolo client con i pixel disposti in una matrice
# 28x28 il dataset viene restitutito diviso in batch
def preprocess(dataset):
  def batch_format_fn(element):
      return (tf.reshape(float(element['pixels']/255), [-1, 28, 28, 1]),
              tf.reshape(element['label'], [-1, 1]))
  return dataset.batch(BATCH_SIZE).map(batch_format_fn)


In [4]:
# Funzione per la creazione di un dataset ClientData a partire dal dataset di training a cui viene
# aggiunta una colonna client_nums che assegna ad ogni riga un client randomico
def create_clients(dataset):
    # Viene creata una lista randomica di client
    client_nums = list(range(NUM_CLIENTS))
    generator = np.random.default_rng(42)
    clients = generator.choice(client_nums, len(dataset))
    dataset['client_nums'] = clients

    # Viene convertito il dataset in dizionari, uno per ogni client, con label e pixel associati
    client_train_dataset = collections.OrderedDict()
    grouped_dataset = dataset.groupby('client_nums')
    for key, item in grouped_dataset:
        current_client = grouped_dataset.get_group(key)
        data = collections.OrderedDict((('label',current_client.iloc[:,0]), ('pixels', current_client.iloc[:,1:-1])))
        client_train_dataset[key] = data

    # I dizionari vengono convertiti in ClientDataset
    def serializable_dataset_fn(client_id):
        client_data = client_train_dataset[client_id]
        return tf.data.Dataset.from_tensor_slices(client_data)

    tff_train_data = tff.simulation.datasets.ClientData.from_clients_and_tf_fn(
        client_ids=list(client_train_dataset.keys()),
        serializable_dataset_fn=serializable_dataset_fn
    )

    return tff_train_data

# Funzione per la creazione di un dataset ClientData a partire dal dataset di test
# Il test è centralizzato, quindi viene impostato un client fittizio a zero
def create_test(dataset):
    zeros = [0]*len(dataset)
    dataset['client_nums'] = zeros

    # Viene convertito il dataset in un dizionario
    client_train_dataset = collections.OrderedDict()
    grouped_dataset = dataset.groupby('client_nums')
    current_client = grouped_dataset.get_group(0)
    data = collections.OrderedDict((('label',current_client.iloc[:,0]), ('pixels', current_client.iloc[:,1:-1])))
    client_train_dataset[0] = data

    # Il dizionario viene convertito in ClientDataset
    def serializable_dataset_fn(client_id = 0):
        client_data = client_train_dataset[0]
        return tf.data.Dataset.from_tensor_slices(client_data)

    tff_train_data = tff.simulation.datasets.ClientData.from_clients_and_tf_fn(
        client_ids=[0],
        serializable_dataset_fn=serializable_dataset_fn
    )

    return tff_train_data

In [7]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

def print_gpu_memory():
    gpus = get_available_gpus()
    for gpu in gpus:
        gpu_name = gpu.split(':')[-1]
        gpu_mem = tf.config.experimental.get_memory_info(gpu)
        print(f"GPU {gpu_name}:")
        print(f"  Memoria libera: {gpu_mem['current'] / (1024 ** 2):.2f} MB")
        print(f"  Memoria totale: {gpu_mem['peak'] / (1024 ** 2):.2f} MB")

print_gpu_memory()

GPU 0:
  Memoria libera: 531.83 MB
  Memoria totale: 531.83 MB


2024-09-02 13:03:21.315189: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /device:GPU:0 with 1743 MB memory:  -> device: 0, name: NVIDIA GeForce MX230, pci bus id: 0000:02:00.0, compute capability: 6.1


In [8]:
# Creazione della lista contenente i client con i relativi dataset
client_data_df = create_clients(train_df)
client_ids = sorted(client_data_df.client_ids)[:ACTIVE_CLIENTS]
federated_train_data = [preprocess(client_data_df.create_tf_dataset_for_client(x)) for x in client_ids]

# Creazione del dataset di test
central_test_df = create_test(test_df)
central_test_df = preprocess(central_test_df.create_tf_dataset_for_client(0))

La struttura del train è la seguente:
    * federated_train_data ha un entry per client
    * Ogni client ha un certo numero di batch
    * Ogni batch è un array con due elementi, uno contiene tutti i pixel, l'altro tutte le label

In [ ]:
# Controlli consistenza datast vs. DataClient

print('Numero di clients: '+str(len(client_data_df.client_ids)))
total = 0
for x in client_data_df.client_ids:
    num_elem = 0
    for i in federated_train_data[x]:
        num = len(list(i[1]))
        num_elem += num
        total += num
    print('Numero di batch per client {}: {}\nNumero elementi per client: {}'.format(x, str(len(federated_train_data[x])), str(num_elem)))
print('TOT TRAIN CD: {} \nTOT TRAIN DF: {}'.format(total, train_df.shape))


num_elem = 0
for i in central_test_df:
    num = len(list(i[1]))
    num_elem += num
print('Numero di batch per il test set: {}'.format(str(len(central_test_df))))
print('TOT TEST CD: {} \nTOT TEST DF: {}'.format(str(num_elem), test_df.shape))

In [ ]:
# Creazione del modello con le API di Keras
def create_keras_model():
  return keras.models.Sequential([keras.layers.Conv2D(filters=32, kernel_size=(5,5), activation='tanh', input_shape=(28, 28, 1), kernel_initializer="glorot_normal"),
                                  keras.layers.AveragePooling2D(pool_size=(2, 2), strides=2),
                                  keras.layers.Conv2D(filters=48, kernel_size=(5,5), activation='tanh'),
                                  keras.layers.AveragePooling2D(pool_size=(2, 2), strides=2),
                                  keras.layers.Conv2D(filters=64, kernel_size=(5,5), padding='same', activation='tanh'),
                                  keras.layers.Flatten(),
                                  keras.layers.Dense(120, activation='tanh', kernel_regularizer=keras.regularizers.l2(0.01)),
                                  keras.layers.Dense(84, activation='tanh', kernel_regularizer=keras.regularizers.l2(0.01)),
                                  keras.layers.Dense(27, activation='softmax')
                                ])
keras_model = create_keras_model()
# Creazione del modello TFF a partire dal modello Keras
tff_model = tff.learning.models.functional_model_from_keras(keras_model,
                                                            loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(),
                                                            input_spec=federated_train_data[0].element_spec,
                                                            metrics_constructor=collections.OrderedDict(accuracy=tf.keras.metrics.SparseCategoricalAccuracy))

In [ ]:
#Al client update si può aggiungere un parametro che indica il numero di epoche in cui ripetere l'addestramento prima di inviare i pesi al server
@tf.function
def client_update(model, dataset, initial_weights, client_optimizer):

  client_weights = initial_weights.trainable
  optimizer_state = client_optimizer.initialize(tf.nest.map_structure(tf.TensorSpec.from_tensor, client_weights))
  for _ in range(10):
    for batch in dataset:
      x, y = batch
      with tf.GradientTape() as tape:
        tape.watch(client_weights)
        # Compute a forward pass on the batch of data
        outputs = model.predict_on_batch(model_weights=(client_weights, ()), x=x, training=True)
        loss = model.loss(output=outputs, label=y)

      # Compute the corresponding gradient
      grads = tape.gradient(loss, client_weights)

      # Apply the gradient using a client optimizer.
      optimizer_state, client_weights = client_optimizer.next(optimizer_state, weights=client_weights, gradients=grads)

  return tff.learning.models.ModelWeights(client_weights, non_trainable=())

@tf.function
def server_update(model, mean_client_weights):
  del model
  return mean_client_weights

In [ ]:
# La logica computazionale di tensorflow federated dev'essere separata rispetto alla logica
# computazionale di tensorflow, qui vengono definite le funzioni di inizializzazione del server
# e di aggiornamento dei client e del server

# Inizializzazione del server con i pesi iniziali del modello
@tff.tensorflow.computation
def server_init():
  return tff.learning.models.ModelWeights(*tff_model.initial_weights)

# Vengono salvati i tipi di dato dei pesi del modello e del dataset
model_weights_type = server_init.type_signature.result
tf_dataset_type = tff.SequenceType(tff.types.tensorflow_to_type(tff_model.input_spec))

# Funzione di aggiornamento del client, viene passato il dataset del client edi pesi
# aggiornati dal server, restituisce i pesi aggiornati del client
@tff.tensorflow.computation(tf_dataset_type, model_weights_type)
def client_update_fn(tf_dataset, server_weights):
  client_optimizer = tff.learning.optimizers.build_adam(learning_rate=0.01)
  return client_update(tff_model, tf_dataset, server_weights, client_optimizer)

# Funzione di aggiornamento del server, riceve i pesi mediati dai client e restituisce
# i pesi aggiornati del server
@tff.tensorflow.computation(model_weights_type)
def server_update_fn(mean_client_weights):
  return server_update(tff_model, mean_client_weights)

In [ ]:
# Aggiornati i tipi di dato dei pesi del modello e del dataset con i tipi federati
# includendo oltre al tipo di dato il placement
federated_server_type = tff.FederatedType(model_weights_type, tff.SERVER)
federated_dataset_type = tff.FederatedType(tf_dataset_type, tff.CLIENTS)

# Definizione della computazione federata per l'inizializzazione del server
# la funzione ritorna i pesi iniziali del modello
@tff.federated_computation
def initialize_fn():
  return tff.federated_eval(server_init, tff.SERVER)

# Definizione della computazione federata per un round di training. Si divide in 3 parti:
# 1. Broadcast dei pesi del server ai client
# 2. Chiamata della funzione di aggiornamento del client
# 3. Il server aggiorna i pesi facendo la media dei pesi dei client
@tff.federated_computation(federated_server_type, federated_dataset_type)
def next_fn(server_weights, federated_dataset):
  server_weights_at_client = tff.federated_broadcast(server_weights)
  client_weights = tff.federated_map(client_update_fn, (federated_dataset, server_weights_at_client))
  server_weights = tff.federated_map(server_update_fn, tff.federated_mean(client_weights))

  return server_weights

In [ ]:
def evaluate(model_weights):
  keras_model = create_keras_model()
  keras_model.compile(
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
  )
  model_weights.assign_weights_to(keras_model)
  keras_model.evaluate(central_test_df)

In [ ]:
# Viene creato l'iterative process con le funzioni init e next custom
federated_algorithm = tff.templates.IterativeProcess(initialize_fn=initialize_fn, next_fn=next_fn)

# Inizializzazione del server
server_state = federated_algorithm.initialize()

In [ ]:
i = 1
for _ in range(5):
    print('pre server state '+str(i))
    server_state = federated_algorithm.next(server_state, federated_train_data)
    print('post server state '+str(i))
    i+=1
evaluate(server_state)